In [1]:
#Project pipeline
# Text cleaning ----- ??
# Coreference resolution  ------ To be done?
# Named entity recognition ---- How to extract the missing ones?

# Entity linking ----
# Relation extraction

In [1]:
# import matplotlib.pyplot as plt
from collections import Counter
# import nltk
# nltk.download('reuters')
from nltk.corpus import reuters
import re
import spacy
nlp = spacy.load('en_core_web_lg') #disable=['parser', 'tagger'])
reuters_fileids_crude = reuters.fileids(categories=['crude'])



# reuters_nlp = [nlp(re.sub('\s+',' ', reuters.raw(i).strip())) for i in reuters_fileids[:100]]
reuters_nlp_crude = [nlp(re.sub('\s+',' ', reuters.raw(i).strip())) for i in reuters_fileids_crude]
len(reuters_nlp_crude)#[0]

# label_counter = Counter()
# reuters.words()
# reuters.sents()
# doc = reuters_nlp[50]

578

In [18]:
reuters_nlp_crude[0]

JAPAN TO REVISE LONG-TERM ENERGY DEMAND DOWNWARDS The Ministry of International Trade and Industry (MITI) will revise its long-term energy supply/demand outlook by August to meet a forecast downtrend in Japanese energy demand, ministry officials said. MITI is expected to lower the projection for primary energy supplies in the year 2000 to 550 mln kilolitres (kl) from 600 mln, they said. The decision follows the emergence of structural changes in Japanese industry following the rise in the value of the yen and a decline in domestic electric power demand. MITI is planning to work out a revised energy supply/demand outlook through deliberations of committee meetings of the Agency of Natural Resources and Energy, the officials said. They said MITI will also review the breakdown of energy supply sources, including oil, nuclear, coal and natural gas. Nuclear energy provided the bulk of Japan's electric power in the fiscal year ended March 31, supplying an estimated 27 pct on a kilowatt/hour 

In [7]:
def clean(article:str):
    pattern = re.compile("\n")
    article = re.sub(pattern,"",article)
    pattern = re.compile(" +")
    article = re.sub(pattern," ",article)
    pattern = re.compile("(\s&lt;.{1,4}>)")
    article = re.sub(pattern,"",article)
    # pattern = re.compile(".'s")
    # article = re.sub(pattern,"'s",article) #???
    article = article.strip()
    #re.findall("(\w[^.]*.)",article)
    return article

In [27]:
import neuralcoref
neuralcoref.add_to_pipe(nlp)

cleaned_text = []
for i in reuters_fileids_crude:
    text = reuters.raw(i).strip() 
    # article = nlp(text)
    # print(article)
    c_text = nlp(clean(text))
    # cleaned_text.append(c_text)
    resolved_text = nlp(c_text._.coref_resolved)
resolved_text    

ValueError: spacy.strings.StringStore size changed, may indicate binary incompatibility. Expected 112 from C header, got 64 from PyObject

In [11]:
reuters_text = [reuters.raw(i).strip() for i in reuters_fileids_crude]
text = reuters_text[25].replace("\n", " ").strip()
doc = nlp(text)

In [10]:
# doc = reuters_nlp_crude[25]
# len(doc)
# doc
# [len(reuters_nlp[i]) for i in reuters_fileids(categories=['fuel'])] #reuters_fileids(categories=['fuel']

# len(reuters.paras())
# reuters.paras()[5]

In [12]:
print(*nlp.pipeline, sep='\n')

('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec object at 0x7f3089cee6b0>)
('tagger', <spacy.pipeline.tagger.Tagger object at 0x7f3089cee770>)
('parser', <spacy.pipeline.dep_parser.DependencyParser object at 0x7f30c761c360>)
('attribute_ruler', <spacy.pipeline.attributeruler.AttributeRuler object at 0x7f3089d62320>)
('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer object at 0x7f3089d56640>)
('ner', <spacy.pipeline.ner.EntityRecognizer object at 0x7f3089eb1130>)


In [13]:
doc

ABU DHABI MARKETING SAID NOT BREACHING OPEC PACT   A senior Abu Dhabi oil official said   in remarks published today the emirate, largest producer in the   United Arab Emirates (UAE), was succeeding in marketing its   crude oil without breaching OPEC accords.       Khalaf al-Oteiba, Marketing Director at the Abu Dhabi   National Oil Co (ADNOC), told the company's Petroleum Community   magazine ADNOC was also keen to keep good customer relations.   "The company will maintain its dialogue with and care for its   customers in accordance with market conditions...And take   necessary steps to guarantee marketing its production," he said.      "The present oil marketing policy of ADNOC is based on   adherence to OPEC decisions of December 1986 to control   production and establish a new pricing system in an attempt to   stabilize the market," he added.       OPEC agreed last December to limit production to 15.8 mln   bpd and return to fixed prices averaging 18 dlrs a barrel.       Oteiba sai

In [19]:
import pandas as pd
list_ent = []
for i in range(len(doc)):
    for ent in doc.ents:
# for ent in doc.ents:
    #check if entity is equal 'LOC' or 'GPE'
        if ent.label_ in ['LOC', 'GPE', 'ORG', 'PERS']:
            if ent.text not in list_ent:
                list_ent.append([ent.text, ent.label_])
# list_ent
df = pd.DataFrame(list_ent, columns=["entity", "type"])
# df.groupby('type').value_counts()
df['type'].value_counts()
df.head(10)


,entity,type
0,ABU DHABI MARKETING,ORG
1,OPEC,ORG
2,Abu Dhabi,ORG
3,the United Arab Emirates,GPE
4,OPEC,ORG
5,the Abu Dhabi National Oil Co,ORG
6,ADNOC,ORG
7,Petroleum Community,ORG
8,ADNOC,ORG
9,ADNOC,ORG


In [ ]:
df.to_csv('data/list_ent.csv')

In [ ]:
def reset_pipeline(nlp, pipes):
    # remove all custom pipes
    custom_pipes = [pipe for (pipe, _) in nlp.pipeline
                    if pipe not in ['tagger', 'parser', 'ner',
                                    'tok2vec', 'attribute_ruler', 'lemmatizer']]
    for pipe in custom_pipes:
        _ = nlp.remove_pipe(pipe)
    # re-add specified pipes
    for pipe in pipes:
        if 'neuralcoref' == pipe or 'neuralcoref' in str(pipe.__class__):
            nlp.add_pipe(pipe, name='neural_coref')
        else:
            nlp.add_pipe(pipe)

    print(f"Model: {nlp.meta['name']}, Language: {nlp.meta['lang']}")
    print(*nlp.pipeline, sep='\n')
    
reset_pipeline(nlp, ['init_coref'])

In [ ]:
reset_pipeline(nlp, ['lemmatizer'])

## Relationship extraction (Vanessa)

In [ ]:
from spacy.tokens import Token
from spacy import Language

Token.set_extension('ref_n', default='', force = True)
Token.set_extension('ref_t', default='', force = True)

@Language.component("init_coref")
def init_coref(doc):
    for e in doc.ents:
        if e.label_ in ['ORG', 'GOV', 'PERSON']:
            e[0]._.ref_n, e[0]._.ref_t = e.text, e.label_
    return doc

def reset_pipeline(nlp, pipes):
    # remove all custom pipes
    custom_pipes = [pipe for (pipe, _) in nlp.pipeline
                    if pipe not in ['tagger', 'parser', 'ner',
                                    'tok2vec', 'attribute_ruler', 'lemmatizer']]
    for pipe in custom_pipes:
        _ = nlp.remove_pipe(pipe)
    # re-add specified pipes
    for pipe in pipes:
        if 'neuralcoref' == pipe or 'neuralcoref' in str(pipe.__class__):
            nlp.add_pipe(pipe, name='neural_coref')
        else:
            nlp.add_pipe(pipe)

    print(f"Model: {nlp.meta['name']}, Language: {nlp.meta['lang']}")
    print(*nlp.pipeline, sep='\n')
    
reset_pipeline(nlp, ['init_coref'])

## Using dependency trees (From Vanessa)

In [ ]:
options = {"bg": "white", "distance": 130,
           "color": "black", "font": "Source Sans Pro"}
for sent in doc.sents:
    displacy.render(sent, style="dep", options=options)
    # displacy.render(doc, style="dep", options=options)

In [ ]:
# Actually we search for the shortest path between the
# subject running through our predicate (verb) to the object.
# subject and object are organizations in our examples.

# Here are the three helper functions omitted in the book:
# - bfs: breadth first searching the closest subject/object 
# - is_passive: checks if noun or verb is in passive form
# - find_subj: searches left part of tree for subject
# - find_obj: searches right part of tree for object

from collections import deque

def bfs(root, ent_type, deps, first_dep_only=False):
    """Return first child of root (included) that matches
    ent_type and dependency list by breadth first search.
    Search stops after first dependency match if first_dep_only
    (used for subject search - do not "jump" over subjects)"""
    to_visit = deque([root]) # queue for bfs

    while len(to_visit) > 0:
        child = to_visit.popleft()
        # print("child", child, child.dep_)
        if child.dep_ in deps:
            if child._.ref_t == ent_type:
                return child
            elif first_dep_only: # first match (subjects)
                return None
        elif child.dep_ == 'compound' and \
             child.head.dep_ in deps and \
             child._.ref_t == ent_type: # check if contained in compound
            return child
        to_visit.extend(list(child.children))
    return None

def is_passive(token):
    if token.dep_.endswith('pass'): # noun
        return True
    for left in token.lefts: # verb
        if left.dep_ == 'auxpass':
            return True
    return False

def find_subj(pred, ent_type, passive):
    """Find closest subject in predicates left subtree or
    predicates parent's left subtree (recursive).
    Has a filter on organizations."""
    for left in pred.lefts:
        if passive: # if pred is passive, search for passive subject
            subj = bfs(left, ent_type, ['nsubjpass', 'nsubj:pass'], True)
        else:
            subj = bfs(left, ent_type, ['nsubj'], True)
        if subj is not None: # found it!
            return subj
    if pred.head != pred and not is_passive(pred): 
        return find_subj(pred.head, ent_type, passive) # climb up left subtree
    else:
        return None

def find_obj(pred, ent_type, excl_prepos):
    """Find closest object in predicates right subtree.
    Skip prepositional objects if the preposition is in exclude list.
    Has a filter on organizations."""
    for right in pred.rights:
        obj = bfs(right, ent_type, ['dobj', 'pobj', 'iobj', 'obj', 'obl'])
        if obj is not None:
            if obj.dep_ == 'pobj' and obj.head.lemma_.lower() in excl_prepos: # check preposition
                continue
            return obj
    return None

def extract_rel_dep(doc, pred_name, pred_synonyms, excl_prepos=[]):
    for token in doc:
        if token.pos_ == 'VERB' and token.lemma_ in pred_synonyms:
            pred = token
            passive = is_passive(pred)
            subj = find_subj(pred, 'ORG', passive)
            if subj is not None:
                obj = find_obj(pred, 'ORG', excl_prepos)
                if obj is not None:
                    if passive: # switch roles
                        obj, subj = subj, obj
                    yield ((subj._.ref_n, subj._.ref_t), pred_name, 
                           (obj._.ref_n, obj._.ref_t))

In [ ]:
# text = """Fujitsu said that Schlumberger Ltd has arranged 
# to buy its stake in Fairchild Inc."""
text = """Iraq said today its troops were pushing Iranian forces out of positions they had initially 
occupied when they launched a new offensive near the southern port of Basra early yesterday."""
doc = nlp(text)
print(*extract_rel_dep(doc, 'push', ['pushing']), sep='\n')

In [ ]:
print(*extract_rel_dep(doc, 'sell', ['sells']), sep='\n')

## Collects Chunks from each sentence

In [ ]:
chunks_list = []
# for i in range(len(doc)):
for sent in doc.sents:
    for chunk in doc.noun_chunks:
        if chunk.text not in chunks_list:
            chunks_list.append([chunk.text, chunk.root.text , chunk.root.dep_,
            chunk.root.head.text])
            # print(chunk.text +'---', chunk.root.text +'---', chunk.root.dep_,
            #     chunk.root.head.text)

In [ ]:
# chunks_list to csv
df = pd.DataFrame(chunks_list, columns=["text", "root Text", "root dep", "head text"])
# df.groupby('type').value_counts()
# df['type'].value_counts()
df.head(10)
df.to_csv('data/chunks_list.csv')

## Coreference

In [41]:
import spacy
import neuralcoref
from spacy import displacy

neuralcoref.add_to_pipe(nlp)

ValueError: spacy.strings.StringStore size changed, may indicate binary incompatibility. Expected 112 from C header, got 64 from PyObject

In [ ]:
doc2 = nlp(doc._.coref_resolved)
displacy.render(doc, style="ent")
print('*****' '\n\n')
displacy.render(doc2, style="ent")

### Jaal

In [75]:
from jaal import Jaal
import pandas as pd
edge_df = pd.read_csv("data/edges.csv")
node_df = pd.read_csv("data/nodes.csv")

edge_df.drop(columns = 'Unnamed: 0', inplace =True) 
node_df.drop(columns = 'Unnamed: 0', inplace =True)

# add edge labels
# edge_df.loc[:, 'label'] = edge_df.loc[:, 'relation'].astype(str)

#Run
# Jaal(edge_df, node_df).plot()

# edge_df 
edge_df


# # add edge labels
# edge_df.loc[:, 'label'] = edge_df.loc[:, 'relation'].astype(str)






,from,to,relation
0,GAF,three billion dlrs,offer
1,Texaco,one billion dlr,offer
2,Texaco,one billion dlr,offer
3,OPEC,18 dlrs,offer
4,OPEC,18 dlrs,offer
...,...,...,...
67,PENNZOIL,10.2 billion,offer
68,PENNZOIL,10.2 billion,offer
69,PENNZOIL,10.2 billion,offer
70,PENNZOIL,10.2 billion,offer


In [76]:
edge_df1 = pd.read_csv("data/example_edges.csv")
node_df1 = pd.read_csv("data/example_nodes.csv")
edge_df1
# node_df1
# edge_df.loc[:, 'label'] = edge_df.loc[:, 'relation'].astype(str)
edge_df.to_csv('edge_df.csv')
node_df.to_csv('node_df.csv')

In [1]:
from jaal import Jaal
from jaal.datasets import load_got
import pandas as pd
edge_df = pd.read_csv("data/edge_df.csv")
node_df = pd.read_csv("data/node_df.csv")
edge_df.drop(columns =['Unnamed: 0'], inplace=True)
node_df.drop(columns =['Unnamed: 0'], inplace=True)
node_df.drop_duplicates(inplace = True)
node_df.reset_index(drop=True, inplace=True)
edge_df.drop_duplicates(inplace = True)
edge_df.reset_index(drop=True, inplace=True)

# init Jaal and run server
Jaal(edge_df, node_df).plot(vis_opts={'height': '600px', # change height
                                      'interaction':{'hover': True}, # turn on-off the hover 
                                      'physics':{'stabilization':{'iterations': 100}}}) # define the convergence iteration of network

Parsing the data...Done
Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/home/mokegg/anaconda3/envs/tf/lib/python3.7/site-packages/dash_bootstrap_components/_table.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


OSError: [Errno 98] Address already in use

In [97]:
node_df

,id
0,GAF
1,Texaco
3,OPEC
13,ONGC
14,Aramco
16,PDVSA
17,Arco
19,Statoil
20,BP
22,Congress


In [99]:
# add edge labels
# edge_df.loc[:, 'label'] = edge_df.loc[:, 'relation'].astype(str)
Jaal(edge_df, node_df).plot()

Parsing the data...Done
Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


OSError: [Errno 98] Address already in use

Parsing the data...Done
Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Jan/2022 16:34:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 16:34:40] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 16:34:40] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 16:34:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 16:34:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 16:34:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 16:34:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 16:34:40] "POST /_dash-update-component HTTP/1.1" 200 -


No trigger


In [73]:
# init Jaal and run server
Jaal(edge_df, node_df).plot(vis_opts={'height': '600px', # change height
                                      'interaction':{'hover': True}, # turn on-off the hover 
                                      'physics':{'stabilization':{'iterations': 100}}}) # define the convergence iteration of network

Parsing the data...Done
Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Jan/2022 16:21:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 16:21:16] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 16:21:16] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 16:21:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 16:21:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 16:21:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 16:21:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 16:21:16] "POST /_dash-update-component HTTP/1.1" 200 -


No trigger


127.0.0.1 - - [18/Jan/2022 16:21:21] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 16:21:23] "POST /_dash-update-component HTTP/1.1" 200 -


In [33]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging # from allennlp_models.pretrained import load_predictor
# predictor = load_predictor("coref-spanbert")

# model_url = 'data/coref-spanbert-large-2021.03.10.tar.gz'
# predictor = Predictor.from_path(model_url)

predictor = Predictor.from_path("data/coref-spanbert-large-2021.03.10.tar.gz")


In [39]:
document = "Ujjwal likes to eat food. Because he likes eating, he bikes every lunchtime to get food. he is a nice person."
predictor.predict(document)

{'top_spans': [[0, 0],
  [1, 1],
  [3, 3],
  [7, 7],
  [8, 8],
  [11, 11],
  [12, 12],
  [19, 19],
  [20, 20],
  [21, 23]],
 'antecedent_indices': [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]],
 'predicted_antecedents': [-1, -1, -1, 0, -1, 3, -1, 3, -1, -1],
 'document': ['Ujjwal',
  'likes',
  'to',
  'eat',
  'food',
  '.',
  'Because',
  'he',
  'likes',
  'eating',
  ',',
  'he',
  'bikes',
  'every',
  'lunchtime',
  'to',
  'get',
  'food',
  '.',
  'he',
  'is',
  'a',
  'nice',
  'person',
  '.'],
 'clusters': [[[0, 0], [7, 7], [11, 11], [19, 19]]]}

In [40]:
# but that's how it looks after coreference resolution (notice the possessive!)
predictor.coref_resolved(document)

'Ujjwal likes to eat food. Because Ujjwal likes eating, Ujjwal bikes every lunchtime to get food. Ujjwal is a nice person.'

In [28]:
txt = """ABU DHABI MARKETING SAID NOT BREACHING OPEC PACT   A senior Abu Dhabi oil official said   in remarks published today the emirate, largest producer in the   United Arab Emirates (UAE), was succeeding in marketing its   crude oil without breaching OPEC accords.       Khalaf al-Oteiba, Marketing Director at the Abu Dhabi   National Oil Co (ADNOC), told the company's Petroleum Community   magazine ADNOC was also keen to keep good customer relations.   "The company will maintain its dialogue with and care for its   customers in accordance with market conditions...And take   necessary steps to guarantee marketing its production," he said.      "The present oil marketing policy of ADNOC is based on   adherence to OPEC decisions of December 1986 to control   production and establish a new pricing system in an attempt to   stabilize the market," he added.       OPEC agreed last December to limit production to 15.8 mln   bpd and return to fixed prices averaging 18 dlrs a barrel.       Oteiba said stabilization of the oil market in the future   depended on how much discipline OPEC showed.       Oteiba said last year, when world oil prices dropped, was   ADNOC's most difficult ever but "a practical and flexible   pricing policy was implemented to relate to the changed market   environment."       He said crude oil sales last year jumped to an average   609,000 bpd of which 73 pct was exported. Refined product sales   totalled eight mln metric tonnes, of which 67 pct was exported.       In 1985, ADNOC marketed a total of 476,000 bpd of crude oil   and 7.2 mln tonnes of refined products."""
txt

'ABU DHABI MARKETING SAID NOT BREACHING OPEC PACT   A senior Abu Dhabi oil official said   in remarks published today the emirate, largest producer in the   United Arab Emirates (UAE), was succeeding in marketing its   crude oil without breaching OPEC accords.       Khalaf al-Oteiba, Marketing Director at the Abu Dhabi   National Oil Co (ADNOC), told the company\'s Petroleum Community   magazine ADNOC was also keen to keep good customer relations.   "The company will maintain its dialogue with and care for its   customers in accordance with market conditions...And take   necessary steps to guarantee marketing its production," he said.      "The present oil marketing policy of ADNOC is based on   adherence to OPEC decisions of December 1986 to control   production and establish a new pricing system in an attempt to   stabilize the market," he added.       OPEC agreed last December to limit production to 15.8 mln   bpd and return to fixed prices averaging 18 dlrs a barrel.       Oteiba s

In [32]:
import textacy
pattern = [{'POS':'VERB'}]
for verb_span in textacy.extract.matches.token_matches(txt, patterns=pattern):
    print(verb_span)

verb_list = [verb_span for verb_span in textacy.extract.matches.token_matches(txt, patterns=pattern)]

AttributeError: 'str' object has no attribute 'vocab'

In [30]:
def tagging(txt):
    verb_phrases = textacy.extract.matches.token_matches(txt, patterns=pattern)
    return list(verb_phrases)
tagging(txt)

AttributeError: 'str' object has no attribute 'vocab'

In [26]:
pattern = [{'POS':'VERB'}]
def tagging(txt):
    verb_phrases = textacy.extract.matches.token_matches(txt, patterns=pattern)
    return (verb_phrases)

tagging(txt)

<generator object token_matches at 0x7f2b3e425950>